# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import requests
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv('output_data/city_data.csv').drop(columns='Unnamed: 0')
df.head()

,city,lat,lng,temp,humidity,wind,cloudiness,country,date
0,pa sang,18.5262,98.9394,80.01,94,4.61,40,TH,Sat May 1 09:30:41 2021
1,port alfred,-33.5906,26.8910,62.17,65,2.77,0,ZA,Sat May 1 09:30:42 2021
2,batemans bay,-35.7167,150.1833,57.99,96,1.01,100,AU,Sat May 1 09:30:42 2021
3,butaritari,3.0707,172.7902,82.20,75,9.37,100,KI,Sat May 1 09:30:43 2021
4,victoria,22.2855,114.1577,80.01,80,6.53,80,HK,Sat May 1 09:29:26 2021


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

loc = df[['lat','lng']]
humidity = df.humidity

fig = gmaps.figure(center=(10.0,-5.0),zoom_level=2)
heat_layer=gmaps.heatmap_layer(loc,weights=humidity,dissipating=False,max_intensity=300,point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
df.city.count()

570

In [5]:
df = df.dropna()

In [6]:
df.city.count()

565

In [7]:
hotel_df = df[(df.temp<80)&(df.temp>70)&(df.wind<10)&(df.cloudiness==0)].copy()
hotel_df

,city,lat,lng,temp,humidity,wind,cloudiness,country,date
68,rio grande,-32.0350,-52.0986,78.80,53,2.30,0,BR,Sat May 1 09:26:25 2021
130,saint-philippe,-21.3585,55.7679,77.00,78,4.61,0,RE,Sat May 1 09:27:08 2021
159,rancho grande,-23.5057,-46.3585,75.00,56,6.91,0,BR,Sat May 1 09:32:05 2021
171,linqiong,30.4159,103.4609,75.20,69,4.00,0,CN,Sat May 1 09:32:11 2021
192,saint-pierre,-21.3393,55.4781,77.00,78,4.61,0,RE,Sat May 1 09:31:48 2021
232,port keats,-14.2500,129.5500,75.16,54,5.50,0,AU,Sat May 1 09:32:46 2021
335,ersekvadkert,47.9962,19.2023,75.99,43,3.44,0,HU,Sat May 1 09:33:44 2021
386,ganzhou,28.8184,115.4327,72.61,80,1.41,0,CN,Sat May 1 09:34:12 2021
400,shelomi,33.0747,35.1461,71.60,78,5.75,0,IL,Sat May 1 09:34:20 2021


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df['Hotel Name'] = ''

In [9]:
hotel_df

,city,lat,lng,temp,humidity,wind,cloudiness,country,date,Hotel Name
68,rio grande,-32.0350,-52.0986,78.80,53,2.30,0,BR,Sat May 1 09:26:25 2021,
130,saint-philippe,-21.3585,55.7679,77.00,78,4.61,0,RE,Sat May 1 09:27:08 2021,
159,rancho grande,-23.5057,-46.3585,75.00,56,6.91,0,BR,Sat May 1 09:32:05 2021,
171,linqiong,30.4159,103.4609,75.20,69,4.00,0,CN,Sat May 1 09:32:11 2021,
192,saint-pierre,-21.3393,55.4781,77.00,78,4.61,0,RE,Sat May 1 09:31:48 2021,
232,port keats,-14.2500,129.5500,75.16,54,5.50,0,AU,Sat May 1 09:32:46 2021,
335,ersekvadkert,47.9962,19.2023,75.99,43,3.44,0,HU,Sat May 1 09:33:44 2021,
386,ganzhou,28.8184,115.4327,72.61,80,1.41,0,CN,Sat May 1 09:34:12 2021,
400,shelomi,33.0747,35.1461,71.60,78,5.75,0,IL,Sat May 1 09:34:20 2021,


In [10]:
params={
    'radius':5000,
    'types':'lodging',
    'key':g_key
}

for i,row in hotel_df.iterrows():
    
    url='https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
    params['location'] = f'{row.lat}, {row.lng}'
    res = requests.get(url,params).json()
    hotel_df.loc[i,'Hotel Name'] = res['results'][0]['name']
hotel_df.rename(columns={'city':'City','lat':'Lat','lng':'Lng','country':'Country'},inplace=True)

In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))